In [1]:
import pandas as pd
import numpy as np

In [2]:
path_test_clean = '../data/cleaned_data/test_correct.csv'
test_file= pd.read_csv(path_test_clean)
lnr = test_file['LNR']
test_file.drop(['LNR'], axis=1, inplace=True)

In [3]:
from process_and_ml.pipeline import cat_features_fill_na

test_df_filled = cat_features_fill_na(test_file, cat_features=['CAMEO_DEU_2015', 'OST_WEST_KZ'])

In [4]:
from process_and_ml.pipeline import load_catboost_model

catboost = load_catboost_model('catboost_first.cbm')

In [5]:
#load file with D19 Sociales
from process_and_ml.data_wrangler import load_test_file

test, lnr = load_test_file('../data/train_test_files/Udacity_MAILOUT_052018_TEST.csv')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
d19_df = test['D19_SOZIALES']



In [7]:
y_pred=catboost.predict_proba(test_df_filled)[:, 1]

In [8]:
df_19 = pd.DataFrame(d19_df)
df_19['pred'] = y_pred

In [9]:
df_19 = pd.DataFrame(d19_df)
df_19['pred'] = y_pred

In [1]:
df_19['pred_soc'] = np.where(df_19['D19_SOZIALES']==1, np.where(df_19['pred'] < 0.8, 1, df_19['pred']), df_19['pred'])


NameError: name 'np' is not defined

In [10]:
#BestOption
df_19['pred_soc'] = np.where(df_19['D19_SOZIALES']==1, np.where(df_19['pred'] < 0.5, df_19['pred']*1.8, df_19['pred']), df_19['pred']*0.1)



In [14]:
from helpers.kaggle_submission import kaggle_submission
kaggle_submission(
    column_lnr=lnr,
    y_pred=df_19['pred_soc'].values,
    submission_filename='catboost_six_shoot',
    submission_message='CatBoost trained with augment update weights.')

Successfully submitted to Udacity+Arvato: Identify Customer Segments


100%|██████████| 1.04M/1.04M [00:07<00:00, 141kB/s] 
